In [19]:
import numpy as np

def LU(A):
    U = np.copy(A).astype('float32')
    m, n = A.shape
    L = np.eye(n).astype('float32')
    for k in range(n-1):
        for j in range(k+1,n):
            L[j,k] = U[j,k]/U[k,k]
            U[j,k:n] -= L[j,k] * U[k,k:n]
    return L, U

Next we define in-place LU factorization.

In [26]:
def LU_inplace(A):
    A = A.astype('float32')
    m, n = A.shape
    for k in range(n-1):
        for j in range(k+1,n):
            A[j,k] = A[j,k]/A[k,k]
            A[j,(k+1):n] -= A[j,k] * A[k,(k+1):n]
    return A

Both of these functions allow us to compute the LU factorization. However, where the first creates and stores in memory two matrices, L and U, the second function stores the two matrices in the previously allocated memory address for the input matrix. The ones for L do not need to be stored, they are assumed. Do not use this if you still have use for your input matrix after computing the decomposition.

In [32]:
x = np.array([[1,2,3],[4,5,6],[7,8,9]])

L, U = LU(x)
print(U)
print(L)


[[ 1.  2.  3.]
 [ 0. -3. -6.]
 [ 0.  0.  0.]]
[[1. 0. 0.]
 [4. 1. 0.]
 [7. 2. 1.]]


In [33]:
A = LU_inplace(x)
print(A)

[[ 1.  2.  3.]
 [ 4. -3. -6.]
 [ 7.  2.  0.]]


We see that this holds the same information as the original function outputs.

Unfortunately, LU factorization like this is unstable for many matrices. The problem lies in Gaussian elimination. The introduction of partial pivoting results in a more stable algorithm. For each column to be reduced, the element that is largest at or below current eliminate is used. That row and the original row are swapped using a row-swap operation.

In [147]:
def swap(a,b): # used to swap rows in the algorithm.
    temp = np.copy(a)
    a[:] = b
    b[:] = temp
    return a,b

def LU_pivot(A):
    U = np.copy(A).astype('float32')
    m,n = A.shape
    
    L = np.eye(n).astype('float32')
    P = np.eye(n).astype('float32')
    
    for k in range(n-1):
        i = k
        for l in range(k+1,m):
            if abs(U[l,k]) > abs(U[i,k]):            # check magnitudes of elements from U[k,k] to U[m,k]
                i = l
                
        U[k,k:n], U[i,k:n] = swap(U[k,k:n],U[i,k:n]) # apply partial pivoting by swapping rows in U and L
        L[k,0:k], L[i,0:k] = swap(L[k,0:k],L[i,0:k])
        
        for j in range(k+1,m):                       # original gaussian elimination algorithm, applied for each column after
            L[j,k] = U[j,k] / U[k,k]                 # pivoting for that column is performed.
            U[j,k:n] -= L[j,k] * U[k,k:n]
        
            
    return L, U
    
    
    
    

In [148]:
A = np.array([[2,1,1,0],
             [4,3,3,1],
             [8,7,9,5],
             [6,7,9,8]])
L, U = LU_pivot(A)

In [149]:
print(L)

[[ 1.          0.          0.          0.        ]
 [ 0.75        1.          0.          0.        ]
 [ 0.5        -0.2857143   1.          0.        ]
 [ 0.25       -0.42857143  0.33333334  1.        ]]


In [150]:
print(U)

[[ 8.          7.          9.          5.        ]
 [ 0.          1.75        2.25        4.25      ]
 [ 0.          0.         -0.8571428  -0.28571427]
 [ 0.          0.          0.          0.6666666 ]]
